# Линейные системы.

## Пролог

Предположим, мы проводим некоторый эксперемент в котором участвует $n$ человек(наблюдения), также мы собираем некоторые характеристики(атрибуты) об этих людях (пусть их количество будет $m$). Таким образом $i$-ого человека можно описать вектором $a_i = (a_{i1}, a_{i2}, \dots, a_{im})$, тогда уместить все наблюдения в компактной форме для глаз и написания можно в матричной форме, где $a_i$ будут являться строчками данной матрицы $A$, где $a_{ij}$ - элемент $i$-ой строчки (наблюдения), $j$-ого столбца. 
$\newline$
$$A = \begin{bmatrix}
    a_{11}       & a_{12} & a_{13} & \dots & a_{1m} \\
    a_{21}       & a_{22} & a_{23} & \dots & a_{2m} \\
    \vdots \\
    a_{n1}       & a_{n2} & a_{n3} & \dots & a_{nm}
\end{bmatrix}$$

Чаще всего нам будет удобно использовать обозначение $a_i$ для обозначение вектора столбца длины $n \times  1$ $i$-го атрибута. В этом случае например линейную модель можно записать ввиде: $b = x_1 a_1 + \dots x_m a_m$ (что эквивалентно матриченой форме $Ax = b$, где $x = (x_1, \dots, x_m)^T, A = (a_1, \dots, a_m)$). Размерности $A \in \mathbb{R}^{n \times m}, b \in \mathbb{R}^{n \times 1}, x \in \mathbb{R}^{m \times 1}$.

### Где встречается

* Линейная регрессия
* Задачи оптимизации

### Задача

$$
Ax = b
$$

Дано: матрица $A_{n\times m}$, вектор $b_{n\times1}$

Найти: вектор $x_{m\times1}$

### Существование решения

#### Теорема Кронекера-Капелли
Система уравнений называется совместной, когда:

$$
\textbf{rank}(A) = \textbf{rank}(A|B)
$$

1. Если система несовместна, то решений **не существует**.
2. Если система совместна и $\textbf{rank}(A) = n$ - существует **единственное** решение.
3. Если система совместна и  $\textbf{rank}(A) < n$ - существует **бесконечное** количество решений.

Таким образом, когда $\textbf{rank}(A) = n$ матрица $A$ имеет полный ранг и решение существует.

Также будем называть, когда $n = m$ - определенная система, $n > m$ - переопределенная система (в общем случае решений не имеет), $n < m$ - недоопределенная система (решение неединственно).

На этой части рассмотрим базовые алгоритмы на определенных системах.

## Решаем линейные системы

$$
Ax = b \rightarrow x = A^{-1}b
$$

Все.

На самом деле нет. А почему нет?

1. Получить обратную матрицу в общем случае дольше, чем работа других алгоритмов
2. Сложность получения обратной матрицы $O(n^3)$, что сравнимо с альтерантивными алгоритмами, но в случае LU - алгоритм итеративный и при достижении разложения - алгоритм работает за $O(n^2)$
3. Получение обратной матрицы - численно менее стабильный алгоритм 
4. Необходимо хранить обратную матрицу, в то время как итеративные алгоритмы in-place
5. Иногда расчет $A^{-1}$ - оверкилл для получения решения, так как это не учитывает структуру матрицы 


In [34]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from time import time

A = np.random.rand(10_000,10_000)
x = np.random.rand(10_000,1)
b = A@x

start = time()
x_est = np.linalg.inv(A)@b

error_inv = np.linalg.norm(x-x_est)
time_inv = time() - start

x_est_lu = np.linalg.solve(A,b)

error_lu = np.linalg.norm(x-x_est_lu)
time_lu = time() - start - time_inv

lu, piv = lu_factor(A)
x_est_lu_sc = lu_solve((lu, piv), b)

error_lu_sc = np.linalg.norm(x-x_est_lu_sc)
time_lu_sc = time() - start - time_lu - time_inv

pct_inv = error_inv/error_lu_sc
pct_lu = error_lu/error_lu_sc

print(f'Time in sec. inversion: {time_inv:.1f}, LU: {time_lu:.1f}, LU_sci: {time_lu_sc:.1f}')
print(f'Error inv/lu_sc: {pct_inv:.1f} lu/lu_sc: {pct_lu:.1f} ')

Time in sec. inversion: 14.9, LU: 4.2, LU_sci: 4.3
Error inv/lu_sc: 641.6 lu/lu_sc: 4.5 


## Gaussian Elimination

Начнем с того какие системы нам хотелось бы решать, или какие системы решать проще, то есть какого вида должна быть матрица $A$, чтобы система решалась простым методом подставновки.

Напримир треугольная матрица.

### Треугольные матрицы

1. L - Lower triangular matrix (нижне треугольная матрица)


$$ L = 
\begin{bmatrix}
a_{11} & 0 & 0\\
a_{21} & a_{22} & 0\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}
$$

1. U - Upper triangular matrix (верхне треугольная матрица)

$$ U =
\begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
0 & a_{22} & a_{23}\\
0 & 0 & a_{33}
\end{bmatrix}
$$

Тогда, допустим следующая система решалась бы простым исключением переменных:

$$
\begin{align}
\begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
0 & a_{22} & a_{23}\\
0 & 0 & a_{33}
\end{bmatrix} 
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
\end{bmatrix} = 
\begin{bmatrix}
{b_1}\\
{b_2}\\
{b_3}\\
\end{bmatrix}\\
x_{3}a_{33} = b_3 \quad \Rightarrow \quad x_3^{*} = \frac{b_3}{a_{33}} \\
x_{2}a_{22} + x_3^{*}a_{23} = b_2 \quad \Rightarrow \quad x_2^{*} = \frac{b_2 - x_3^{*}a_{23}}{a_{22}} \\
x_{1}a_{11}+ x_2^{*}a_{12} + x_3^{*}a_{13} = b_1 \quad \Rightarrow \quad x_1^{*} = \frac{b_1 - x_2^{*}a_{12} - x_3^{*}a_{13} }{a_{11}}
\end{align}
$$

Таким образом, система $N\times N$ решается за $\frac{N(N-1)}{2}$ вычетаний и умножений и $N$ делений, что похоже на $O(N^2)$

### Gaussian Elimination in matrix forms

Метод исключения Гаусса основан на том, чтобы привести матрицу к такому виду. Метод Гаусса начинается с вычитания первой строки из остальных строк, чтобы занулить первый столбец, тем самым исключая переменную $x_1$ (определяя ее). Затем та же процедура повторяется для остальных столбцов.

Данную процедуру можно описать путем умножения матриц, например:

$$
A = 
\begin{bmatrix}
4 & 4 & 2\\
4 & 5 & 3\\
2 & 3 & 3
\end{bmatrix} 
$$

Чтобы занулить первый столбец надо вычесть из 2 строчки первую 1 раз и 0.5 раз из 3 строчки вычесть первую, в матричной форме это записывается следующим образом.

Если мы точно знаем какие операции нужно проводить для сведения матриц к треугольному (ступенчатому) виду, то можно воспользоваться следующим:
> Домножая матрицу слевой стороны на единичную $\mathbf{I}$, у которой на $(i, j)_{i \neq j}$ позициях находятся элементы , которые обозначают следующие операции: "вычти из $i$ строчки $j$ строчку", если элемент $-m_{ij}$, и сложи строчки если $m_{ij}$

В данной операции на месте $m_{21} = -1$, что означает вычитание из 2 строчки первой

$$
\begin{align}
M_{1}A =
\begin{bmatrix}
1 & 0 & 0\\
-1 & 1 & 0\\
-0.5 & 0 & 1
\end{bmatrix} 
\begin{bmatrix}
4 & 4 & 2\\
4 & 5 & 3\\
2 & 3 & 3
\end{bmatrix}  = 
\begin{bmatrix}
4 & 4 & 2\\
0 & 1 & 1\\
0 & 1 & 2
\end{bmatrix} \\ 
 \newline
M_{2}M_{1}A = 
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & -1 & 1
\end{bmatrix}
\begin{bmatrix}
4 & 4 & 2\\
0 & 1 & 1\\
0 & 1 & 2
\end{bmatrix} = 
\begin{bmatrix}
4 & 4 & 2\\
0 & 1 & 1\\
0 & 0 & 1
\end{bmatrix} = U
\end{align}
$$

В итоге получаем,

$$
M_{2}M_{1}Ax = Ux = M_{2}M_{1}b
$$

Что уже решается делом техники, нужно лишь найти произведение $M_{2}M_{1}b$

Также можно заметить, что $M_{2}M_{1}A = U$ можно переписать в $A = LU$, где $L = M_1^{-1}M_2^{-1}$

$
L = M_1^{-1}M_2^{-1} = \begin{bmatrix}
1 & 0 & 0\\
1 & 1 & 0\\
0.5 & 0 & 1
\end{bmatrix}\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 1 & 1
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0\\
1 & 1 & 0\\
0.5 & 1 & 1
\end{bmatrix}
$

Где все суб-диагональные элементы в $L$ это множители Гаусса, c которыми мы сталкнулись при процедуре исключения переменных. 

**Как правило $LU$ разложение и ассоциируется с методом Гаусса.**

При получении $LU$ разложения остается только лишь дважды решить систему с треугольными матрицами.

## LU разложение

**Теорема. Для любой матрицы $\mathbf{A \in \mathbb{R}^{n\times n}, \mathbb{det}(A(1:k, 1:k))\neq 0, k = 1:(n-1)}$ тогда существует нижнетреугольная матрица $\mathbf{L \in \mathbb{R}^{n\times n}}$ и верхнетреугольная матрица $\mathbf{U \in \mathbb{R}^{n\times n}}$, такие что $\mathbf{A = LU}$, если матрица $\mathbf{A}$ - невырождена, тогда разложение уникально и $\mathbf{\mathbb{det}(A) = u_{11}u_{22}\dots u_{nn}}$.**

* LU разложение считается самым дешевым матричным разложением $O(\frac{2}{3}n^3)$
### LU разложение, right-looking алгоритм 

$$
A = LU \\
A \rightarrow \begin{bmatrix}
\alpha_{11} & \alpha_{12}^{\top} \\
\alpha_{21}^{\top} & A_{22} 
\end{bmatrix}, L \rightarrow \begin{bmatrix}
1 & 0 \\
l_{21} & L_{22} 
\end{bmatrix}, U \rightarrow \begin{bmatrix}
u_{11} & u_{12}^{\top} \\
0 & U_{22} 
\end{bmatrix} \\
\begin{bmatrix}
\alpha_{11} & \alpha_{12}^{\top} \\
\alpha_{21}^{\top} & A_{22} 
\end{bmatrix} = \begin{bmatrix}
u_{11} & u_{12}^{\top} \\
l_{21} & l_{21}u_{12}^{\top} + L_{22}U_{22} 
\end{bmatrix} \\
\begin{cases}
\alpha_{11} = u_{11} \\
a_{12}^{\top} = u_{12}^{\top} \\
\alpha_{21} = u_{11}l_{21} \\
A_{22} - l_{21}u_{12}^{\top} = L_{22}U_{22}
\end{cases} \Rightarrow \begin{cases}
\alpha_{21} \leftarrow \frac{\alpha_{21}}{\alpha_{11}} \\
A_{22} \leftarrow A_{22} - \alpha_{21}\alpha_{12}^{\top}
\end{cases} \rightarrow \begin{cases}
\tilde \alpha \leftarrow \dots \\
A_{22} \leftarrow \dots
\end{cases} \rightarrow 
\begin{cases}
\tilde \alpha \leftarrow \dots \\
A_{nn} \leftarrow \dots
\end{cases}
$$

$\alpha_{21}$ размером $(n-k-1)$, тогда затраты на обновление $\alpha_{21} \leftarrow \frac{\alpha_{21}}{\alpha_{11}}$ - $(n-k-1)$, затраты на само деление мало при росте $n$, $A_{22}$ размером $(n-k-1)\times(n-k-1)$, тогда обновление матрицы требует $2(n-k-1)\times(n-k-1)$ 

$
\sum_{k=0}^{n-1}2(n-k-1)^2 = \sum_{j=0}^{n-1}2j^2 \approx 2\int_{0}^{n}x^2 \,dx = \frac{2}{3}n^3$
* LU - не стабильный алгоритм. 

### PLU
Нам нужны гарантии того, что ведущий элемент ненулевой, мы не делим на ноль.

Например

$
A = \begin{bmatrix}
1 & 1 & 1\\
2 & 2 & 5\\
4 & 6 & 8
\end{bmatrix}, \, M_1 = \begin{bmatrix}
1 & 0 & 0\\
-2 & 1 & 0\\
-4 & 0 & 1
\end{bmatrix}, \, M_1A = \begin{bmatrix}
1 & 1 & 1\\
0 & 0 & 3\\
0 & 2 & 4
\end{bmatrix}, \quad \text{алгоритм ломается, так как придется делить на 0}
$

Либо, $\epsilon=0$

$
A = \begin{bmatrix}
1 & 1 & 1\\
2 & 2 +\epsilon & 5\\
4 & 6 & 8
\end{bmatrix}, \, M_1A = \begin{bmatrix}
1 & 1 & 1\\
0 & \epsilon & 3\\
0 & 2 & 4
\end{bmatrix}, \, M_2 = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & -\frac{2}{\epsilon} & 1
\end{bmatrix}, \, M_2M_1A = \begin{bmatrix}
1 & 1 & 1\\
0 & \epsilon & 3\\
0 & 0 & 4 - \frac{6}{\epsilon}
\end{bmatrix}, \, L = M_1^{-1}М_2^{-1} = \begin{bmatrix}
1 & 0 & 0\\
2 & 1 & 0\\
4 & \frac{2}{\epsilon} & 1
\end{bmatrix}\\
\newline
\epsilon > 0 \, (small) \rightarrow (4 - \frac{6}{\epsilon}) \rightarrow - \frac{6}{\epsilon} \\
\tilde U = \begin{bmatrix}
1 & 1 & 1\\
0 & \epsilon & 3\\
0 & 0 &  - \frac{6}{\epsilon}
\end{bmatrix}, \, \tilde L \tilde U = \begin{bmatrix}
1 & 1 & 1\\
2 & 2 +\epsilon & 5\\
4 & 6 & 4
\end{bmatrix} \neq A
$

А что, если бы мы могли как нибудь поменять местами строчки, например $2\longleftrightarrow3$, данная операция выплняется через следующее матричное умножение:
> умножая на единичную матрицу $(i,j)_{i=j} = 1$ матрица не меняется, умножение в таком виде говорит нам о том, что $i$-ая строчка находится в $j$-ом индексе, но если переставить единицы по строчкам то получится и смена строчек после умножения

Сейчас мы хотим переставить 2 и 3 строчки, тогда $1$ из 2 строчки единичной матрицы должна стоять в 3 столбце, а 1 из 3 строчки единичной матрицы должна стоять во 2 столбце. **Такие матрицы называются матрицами перестановок со свойством** $\mathbf{P^{-1} = P^{\top}}$

$
P_1M_1A = \begin{bmatrix}
1 & 0 & 0\\
0 & 0 & 1\\
0 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
1 & 1 & 1\\
0 & 0 & 3\\
0 & 2 & 4
\end{bmatrix} = \begin{bmatrix}
1 & 1 & 1\\
0 & 2 & 4\\
0 & 0 & 3
\end{bmatrix}
$

Идея состоит в том чтобы на шаге $k$ найти максимальный элемент в столбце под текущей переменной $x_k$ и поменять эту строчку $k$ со строкой, которая содержит $x_{max_k}$

Общий вывод PLU разложения:

$
\begin{aligned}
& \text{Solution of} \, Ax = b, \, \text{where} \\
& PA = LU \\
& \textbf{Step 1.} \, LUx = P^{\top}b \\
& \textbf{Step 2.} \, Ly = \tilde b \\
& \textbf{Step 3.} \, Ux = y\\
& \textbf{Step 4.} \, \tilde x
\end{aligned}
$

$
\begin{aligned}
& \text{Solution of} \, Ax = b, \, \text{where} \\
& PAQ^{\top} = LU \\
& \textbf{Step 1.} \, LUx = P^{\top}b\\
& \textbf{Step 2.} \, Lz = \tilde b \\
& \textbf{Step 3.} \, Uy = z \\
& \textbf{Step 4.} \, \tilde x = Q^{\top}y
\end{aligned}
$

**LU Теорема. Для каждой матрицы полного ранга** $\mathbf{R^{n\times n}}$ **существует матрица перестановок P, такая что матрица PA может быть представлена через LU разложение, где L - нижне тругольная матрица со всеми диагональными элементами 1**

**Доказательство:**

Если матрица полного ранга то с помощью перестановки

$
\begin{aligned}
& P_1A = \begin{bmatrix}
a_{11} & A_{12}\\
A_{21} & A_{22} 
\end{bmatrix},(a_{11}\neq0), P_1A= \begin{bmatrix}
1 & 0\\
\frac{A_{21}}{a_{11}} & \mathbf{I} 
\end{bmatrix}
\begin{bmatrix}
a_{11} & A_{12}\\
0 & S 
\end{bmatrix}\\
& S = A_{22} - a_{11}^{-1}A_{21}A_{12} \\
& \mathbb{det}(A)\neq 0, \mathbb{det}(S)\neq 0, \, \text{продолжаем разложение по индукции для S}
\end{aligned}
$

Где $S$ является дополнением Шура матрицы $A$ к элементу $A_{22}$, дополнение Шура играет ключевую роль в алгоритмических приложениях для решения блочных систем уравнений методом Гаусса/LU разложением.

### Блочный метод Гаусса

Бывает удобным представление выражения метода Гаусса как обновление под-матриц исходной матрицы, на каждом шаге метода Гаусса нижняя замыкающая под-матрица называется **дополнением Шура.**

$$
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix} = \begin{bmatrix}
L_{11} & 0 \\
L_{21} & L_{22}
\end{bmatrix} \begin{bmatrix}
U_{11} & U_{12} \\
0 & U_{22}
\end{bmatrix} = \begin{bmatrix}
L_{11}U_{11} & L_{11}U_{12} \\
L_{21}U_{11} & L_{22}U_{22} + L_{21}U_{12}
\end{bmatrix}
$$

Также иногда записывается в эквивалентной записи:

$$
A \in \mathbb{R}^{n \times n}, A_{11} \in \mathbb{R}^{r \times r}, A_{12} \in \mathbb{R}^{r \times (n-r)},  A_{21} \in \mathbb{R}^{(n-r) \times r}, A_{22} \in \mathbb{R}^{(n-r) \times (n-r)}\\
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix} = \begin{bmatrix}
L_{11} & 0 \\
L_{21} & \mathbf{I}_{n-r}
\end{bmatrix}\begin{bmatrix}
\mathbf{I}_{r} & 0 \\
0 & S
\end{bmatrix}\begin{bmatrix}
U_{11} & U_{12} \\
0 & \mathbf{I}_{n-r}
\end{bmatrix}
$$

$
U_{12} = L_{11}^{-1}A_{12} \\
L_{21} = A_{21}U_{11}^{-1} \\ 
L_{22}U_{22} = A_{22} - L_{21}U_{11}^{-1}L_{11}^{-1}A_{12} = A_{22} - A_{21}A_{11}^{-1}A_{12} \\
S =  A_{22} - A_{21}A_{11}^{-1}A_{12} - \, \text{Блочный вариант обновления в методе Гаусса}
$

Либо еще одно свойство, если $det(A), det(A_{11}) \neq 0$

$
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix} \begin{bmatrix}
X \\
S^{-1} 
\end{bmatrix} = \begin{bmatrix}
0 \\
\mathbf{I} 
\end{bmatrix}
$

```python
def block_lu_decomposition(A):
    pass
```

### Ремарка про существование и единственность LU разложения 

Как правило за существование и единственность $LU$ разложения отвечает сингулярность матрицы и равенство нулю ведущих миноров. Что вообще означает уникальность и в чем оно проявляется? Тут пример.

$
\begin{bmatrix}
2 & 1 & 3 \\
1 & 4 & 0 \\
3 & 5 & 3
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 \\
0.5 & 1 & 0 \\
1.5 & 1 & 1
\end{bmatrix}\begin{bmatrix}
2 & 1 & 3 \\
0 & 3.5 & -1.5 \\
0 & 0 & 0
\end{bmatrix}
$

Это $LU$ разложение матрицы с линейно-зависимой 3 строчкой, сумма первой и второй строчки равна третьей, следовательно матрица не обладает полным рангом, ранг = 2, а в матрице $U$ на диагонали появляется нулевой элемент, таким образом мы можем назначить $l_{33} = *$ - абсолютно любым. Таким образом разложение для данной матрицы не существует единственного разложения. 

$LU$ - разложение является численным алгоритмом получения произведения треугольных матриц, например можно разложить:

$
\begin{bmatrix}
1 & 1  \\
1 & 1 
\end{bmatrix} = \begin{bmatrix}
1 & 0  \\
1 & 1 
\end{bmatrix}\begin{bmatrix}
1 & 1  \\
0 & 0 
\end{bmatrix}
$

$
\begin{bmatrix}
0 & 0  \\
0 & 0 
\end{bmatrix} = \begin{bmatrix}
1 & 0  \\
0 & 1 
\end{bmatrix}\begin{bmatrix}
0 & 0  \\
0 & 0 
\end{bmatrix}
$

### LU на прямоугольных матрицах

Нетрудно показать, что LU разложение может быть произведено и над матрицами $A \in \mathbb{R}^{n \times r}, n > r$

$
\begin{bmatrix}
1 & 2 \\
3 & 4 \\
5 & 6 
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
3 & 1 \\
5 & 2 
\end{bmatrix}\begin{bmatrix}
1 & 2 \\
0 & -2 
\end{bmatrix}
$

$
A \in \mathbb{R}^{r \times n}, r < n \\
\begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
4 & 1 
\end{bmatrix}\begin{bmatrix}
1 & 2 & 3\\
0 & -3 & -6
\end{bmatrix}
$

### Parallel LU
> Блок о том как считать блочный LU параллельно 

### LU для симметричных матриц

Если матрица симметричная, то существует связь между матрицами $L$ и $U$, например:

$
\begin{bmatrix}
a & b \\
c & d 
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
\frac{c}{a} & 1 
\end{bmatrix} \begin{bmatrix}
a & c \\
0 & d - (\frac{c}{a})c 
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
\frac{c}{a} & 1 
\end{bmatrix}(\begin{bmatrix}
a & 0 \\
0 & d - (\frac{c}{a})c 
\end{bmatrix} \begin{bmatrix}
1 & \frac{c}{a} \\
0 & 1 
\end{bmatrix})
$

Получается, что $U$ это скалированная построчно $L^{\top}$

### LDL разлоложение 

**Теорема. Eсли $\mathbf{A \in R^{n\times n}}$ матрица симметрична и ее главные миноры не вырождены,  то существует такая нижне треугольная матрица L и диагональная матрица D, что**

$\mathbf{A = LDL^{\top}}$

**При этом разложение единственно**

Существует классическое **скалярное произведение** $<x, y> = x^{\top}y$, также можно вывести его обощенную форму $<x, y> = x^{\top}Sy$, $S$ - единичная матрица $\mathbf{I}_n$

Если применить **процесс Грамма-Шмидта** по преобразованию **базиса** $(e_1,e_2, \dots, e_n) \rightarrow (v_1,v_2, \dots, v_n)$ который ортогонален относительно операции обобщенного скалярного произведения, то мы получим $LDL^{\top}$ разложение, в общем, данное разложение является процесс Грамма-Шмидта примененного к скалярному произвдеению.

Пример:

$$
\begin{align}
S &= \begin{bmatrix}
4 & 1 \\
1 & 3
\end{bmatrix} \\
e = (e_1, e_2) &= \begin{pmatrix} \begin{pmatrix} 1 \\ 0 \end{pmatrix} \begin{pmatrix} 0 \\ 1 \end{pmatrix} \end{pmatrix} \\
v_1 &= e_1 \\
v_2 &= e_2 - \frac{<e_2, v_1>}{<v_1, v_1>}v_1 \\
<e_2, v_1> = e_2^{\top}Sv_1 &= \begin{pmatrix} 0 & 1 \end{pmatrix} \begin{bmatrix}
4 & 1 \\
1 & 3
\end{bmatrix}\begin{pmatrix} 1 \\ 0 \end{pmatrix} = 1 \\
<v_1, v_1> = v_1^{\top}Sv_1 &= \begin{pmatrix} 1 & 0 \end{pmatrix} \begin{bmatrix}
4 & 1 \\
1 & 3
\end{bmatrix}\begin{pmatrix} 1 \\ 0 \end{pmatrix} = 4 \\
v_2 &= \begin{pmatrix} 0 \\ 1 \end{pmatrix}  - \frac{1}{4}\begin{pmatrix} 1 \\ 0 \end{pmatrix} =  \begin{pmatrix} -\frac{1}{4} \\ 1 \end{pmatrix}\\
v = (v_1,v_2)  &= \begin{pmatrix} \begin{pmatrix} 1 \\ 0 \end{pmatrix} \begin{pmatrix} -\frac{1}{4} \\ 1 \end{pmatrix} \end{pmatrix} \\
L_0v = e, L_0 = v^{-1}e &= \begin{bmatrix}
1 & 0 \\
\frac{1}{4} & 1
\end{bmatrix} \\
D_1 &= <v_1, v_1> = 4 \\
D_2 &= <v_2, v_2> = \begin{pmatrix} -\frac{1}{4} & 1 \end{pmatrix} \begin{bmatrix}
4 & 1 \\
1 & 3
\end{bmatrix}\begin{pmatrix} -\frac{1}{4} \\ 1 \end{pmatrix} = \frac{11}{4} \\
S = LDL^{\top} &= \begin{bmatrix}
1 & 0 \\
\frac{1}{4} & 1
\end{bmatrix} \begin{bmatrix}
4 & 0 \\
0 & \frac{11}{4}
\end{bmatrix} \begin{bmatrix}
1 & \frac{1}{4} \\
0 & 1
\end{bmatrix}
\end{align}
$$

Это работает для всех симметричных матриц, для несимметричных работает схожее по смыслу и реализации $\mathbf{LDU}$ разложение

### Разложение Холецкого

Если матрица является симметричной, а еще и положительно определенной тогда в разложениие $A=LDL^{\top}$, матрицы $A, D$ обладают одинаковой **инерцией**, тогда мы можем записать $A=LDL^{\top} = (LD^{\frac{1}{2}})(LD^{\frac{1}{2}})^{\top} = \tilde L\tilde L^{\top}$, а матрица $\tilde L$ называется **Cholesky factor**.

**Теорема. Для любой $\mathbf{A \in \mathbb{R}^{n \times n}}$ - симметричной и положительно определенной матрицы существует уникальная нижнетреугольная матрица $\mathbf{L}$ с положительным диагональными элементами, такая что $\mathbf{A = LL^{\top}}$**

**Теорема. Если матрица положительно определеная то в $\mathbf{LU}$ разложении диагональные элементы матрицы $\mathbf{U}$ положительные.**

Доказательство:

Если вспомнить свойства симметричных матриц, матрицы $A$ и $B^{\top}AB$ обладают одинаковой инерцией, тогда в случае положительно определенных матриц $X = B^{\top}AB$ также является положительно определенной. Тогда $A = LU$, $B = (L^{-1})^{\top} = L^{-\top}$, $X = L^{-1}LUL^{-T} = UL^{-\top}$ - положительно определена (положительные элементы на диагонали), следовательно на диагонали у матрицы $U$ также расположены положительные элементы на диагонали это вытекает из того, что матрица $L^{-\top}$ верхнетреугольная матрица с единицами на диагонали и $x_{ii} = u_{ii}$.

### LDL разложение с поворотами

Если симметричная положительно-определенная система плохо приводится к ступенчатому виду, как и в случае с $LU$ разложением мы можем производить умнржение на матрицы, меняя местами строчки или столбцы, если матрица $A$ - симметричная, то матрица $P_1A$ - несимметричная, где матрица $P_1$ - матрица перестановок, c другой стороны матрица $P_1AP_1^{\top}$ - симметричная, что приводит к следующему разложению:

$$
P_1AP_1^{\top} = \begin{bmatrix}
\alpha & v^{\top} \\
v & B
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
\frac{v}{\alpha} & \mathbb{I}_{n-1}
\end{bmatrix}\begin{bmatrix}
\alpha & 0 \\
0 & \tilde A
\end{bmatrix}\begin{bmatrix}
1 & 0 \\
\frac{v}{\alpha} & \mathbb{I}_{n-1}
\end{bmatrix}^{\top}
$$

Где $\tilde A$ - известное нам **дополнение Шура**, $\tilde A = B - \frac{vv^{\top}}{\alpha}$, и проведя рекурссивно разложение и для $\tilde P\tilde A \tilde P^{\top} = \tilde L \tilde D \tilde L^{\top}$ получим общее разложение:

$$
PAP^{\top} = LDL^{\top} \\
P = \begin{bmatrix}
1 & 0 \\
0 & \tilde P
\end{bmatrix}, L = \begin{bmatrix}
1 & 0 \\
\frac{v}{\alpha} & \tilde L
\end{bmatrix}, D = \begin{bmatrix}
\alpha & 0 \\
0 & \tilde D
\end{bmatrix}
$$

## Заключение

1. $A = LU$ - разложение матрицы $A$ с оригинальным расположение строк и столбцов на нижнетреугольную матрицу $L$ и верхнетреугольную матрицу $U$
2. Без правильного упорядочения или перестановок в матрице, разложение может не считаться (технически), так как ведущий диагональный элемент на каждой итерации не должен равняться 0, эта проблема решается домножением на матрицу перестановок строчек на каждой итерации $PA = LU$ либо $PAQ = LU$ перестановкой и строчек и столбцов
3. Любая матрица $A$ допускает $PLU$ разложение. Если матрица $A$ обратима, то сущетсвует $LU$ разложение только тогда, когда все главные миноры матрицы не равны 0.
4. Если $A$ — сингулярная матрица ранга $K$, то она допускает $LU$ -факторизацию первых ведущих $k$ миноров, хотя обратное неверно.
5. Для матрицы $A$ также существует уникальное $A = LDU$ разложение с единицами на главных диагоналях матриц $L,U$ и $D$ - диагональная матрица.  В этом случае $LU$-разложение также будет уникальным, если мы потребуем, чтобы диагональ $L$ или $U$ состояла из единиц. Если матрица $A$ - симметрична, то это разложение превращается в $A = LDL^{\top}$.
6. Если матрица $A$ - симметрична и положительно определена, то существует следующее уникальное разложение, которое называется разложением Холецкого $A = \tilde L \tilde L^{\top}$.
7. $LU$ разложение по сути представляет собой модифицированную форму метода Гаусса.
8. $LU$ разложение используется для решение систем уравнений вида $Ax = b$ в три шага. Получение разложения $A = P^{-1}LU \rightarrow LUx = Pb$. Решение треугольной системы $Ly = Pb$. Решение треугольной системы $Ux = y$. Решение треугольной системы приблизительное можно оценить в $O(n^2)$, $PLU$ - разложение в $O(\frac{2}{3}n^3)$. Следовательно, весь алгоритм решения $Ax = b$ через $PLU$ разложение оценивается в $O(\frac{2}{3}n^3)$.
9. $LU$ разложение может быть неуникальным и для обратимых матриц с ненулевыми главными минорами, если в алгоритме разложения нету ограничение на диагональ матрицы $L$, в базовом $LU$ представлении на диагонали стоят единицы, если мы будем искать решение задачи на всем пространстве, то в полученном разложении всегда можем сделать следующую операцию $\alpha L\frac{1}{\alpha}U$